In [40]:
import json
import numpy as np
import networkx as nx
from scipy.optimize import minimize
from plotter_lib import *

### Definimos la función error para el optimizador

In [41]:
def get_metricas(links, w):

    G = nx.DiGraph()

    for i in range(len(links)):

        n1, n2 = links[i].split(' ')
        G.add_edge(n1, n2, weight=w[i])

    ec = nx.eigenvector_centrality(G, weight="weight")
    bc = nx.betweenness_centrality(G, weight="weight")
    c = nx.clustering(G, weight="weight")

    return [ec, bc, c]

def fobj(pars, args):

    links, players, mu, a = args

    Ec, Bc, C = get_metricas(links, np.array(pars, dtype=int))

    return sum([a[0]*(Ec[player] -mu["ec"][player])**2 + 
               a[1]*(Bc[player] - mu["bc"][player])**2 +
               a[2]*(C[player] - mu["c"][player])**2 for player in players])


#Metricas usadas para optimizar
arx = '../player_metrica_S1.json'
d = json.load(open(arx, "r"))
players_ = [ 'Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10' ]
mu={}
mu["ec"]={  player:np.mean(d[player]["ec"]) for player in players_ }
mu["bc"]={  player:np.mean(d[player]["bc"]) for player in players_ }
mu["c"]={  player:np.mean(d[player]["c"]) for player in players_ }
sd={}
sd["ec"]={  player:np.std(d[player]["ec"]) for player in players_ }
sd["bc"]={  player:np.std(d[player]["bc"]) for player in players_ }
sd["c"]={  player:np.std(d[player]["c"]) for player in players_ }

### Optimizo

In [59]:


# jugador que saco

for p in [1,2,3,4,5,6,7,8,9,10]:

    print ("Player%d ----------------------------------"%p)

    players = players_.copy()
    players.pop(p-1)

    #arxs = "edgelists/red_optima_3_p%d.edgelist"%p


    # todos los posibles links
    n = len(players)
    links = [ ]

    for i in range(n):
        for j in range(n):
            if i!=j:
                links.append( "%s %s"%(players[i],players[j]) )


    # ponderacion a las metricas
    for gamma in [1.5,1.6,1.8,2,2.25,2.5,3,3.25]:

        # minimizador
        print ("\noptimizando ...",gamma)

        a = [ 1.0, 1.07, gamma ]
        args = [ links,players,mu, a  ]
        pars0 = np.ones(len(links))
        bounds = [ (0, 10) for i in range(len(links)) ]
        res = minimize(fobj, pars0, args=args, method="Powell", bounds=bounds)

        print (res.success, res.fun)


        # calculo las metricas para la red optimizada --------------------
        pesos = np.array(res.x, dtype=int)
        Ec,Bc,C = get_metricas(links, pesos)

        # chequeo que esten dentro de los intervalos
        count_no=0

        for player in players:

            l1,l2 = mu["ec"][player]-sd["ec"][player], mu["ec"][player]+sd["ec"][player]
            if Ec[player]>l2 or Ec[player]<l1:
                count_no+=1
            

            l1,l2 = mu["bc"][player]-sd["bc"][player], mu["bc"][player]+sd["bc"][player]
            if Bc[player]>l2 or Bc[player]<l1:
                count_no+=1
            

            l1,l2 = mu["c"][player]-sd["c"][player], mu["c"][player]+sd["c"][player]
            if C[player]>l2 or C[player]<l1:
                count_no+=1

        print ("Casos fuera", count_no)




Player1 ----------------------------------

optimizando ... 1.5


True 0.07393467474162878
Casos fuera 4

optimizando ... 1.6
True 0.06995671739118517
Casos fuera 2

optimizando ... 1.8
True 0.0355475710944355
Casos fuera 2

optimizando ... 2
True 0.08898350303910188
Casos fuera 5

optimizando ... 2.25
True 0.05193027607794397
Casos fuera 3

optimizando ... 2.5
True 0.0542263183573513
Casos fuera 3

optimizando ... 3
True 0.07984326848301225
Casos fuera 5

optimizando ... 3.25
True 0.06026408011488325
Casos fuera 3
Player2 ----------------------------------

optimizando ... 1.5
True 0.09646010187571441
Casos fuera 5

optimizando ... 1.6
True 0.10191574058462566
Casos fuera 4

optimizando ... 1.8
True 0.08291811012775897
Casos fuera 3

optimizando ... 2
True 0.08710588978576739
Casos fuera 4

optimizando ... 2.25
True 0.08865671714036202
Casos fuera 5

optimizando ... 2.5
True 0.04800365124839187
Casos fuera 2

optimizando ... 3
True 0.12945326668302548
Casos fuera 5

optimizando ... 3.25
True 0.06294379922453577
Casos fuera 2
Player3 ----------------

In [1]:
# este dict tiene los valores de gamma por player donde la cantidad de jug afuera es minima
player2gamma = {
    'Player1':1.6,
    'Player2':2.5,
    'Player3':2.5,
    'Player4':3.25,
    'Player5':2.5,
    'Player6':3.0,
    'Player7':3.0,
    'Player8':2.0,
    'Player9':2.5,
    'Player10':2
}

### Plots